# Output Parsers - Padronizando a resposta do modelo

In [11]:
from langchain.prompts import ChatPromptTemplate

In [15]:
review_cliente = """
Este soprador de folhas é bastante incrível. Ele tem quatro configurações: sopro de vela, brisa suave, cidade ventosa e tornado. Chegou em dois dias, bem a tempo para o presente de aniversário da minha esposa. Acho que minha esposa gostanto tanto que ficou sem palavras. Até agora, fui o único a usá-lo, e tenho usando em todas as manhãs alternadas para limpar as folhas do nosso gramado. É um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado, mas acho que vale a penas pelas características extras.
"""

In [16]:
review_template = ChatPromptTemplate.from_template("""
Para o texto a seguir, extraia as seguintes informações:

presente: o item foi comprado para alguém? True caso verdadeiro e False caso falso ou não tenha informação.

dias_entrega: quantos dias para a entrega chegar? se a resposta não for encontrada, retorne -1.

percepcao_de_valor: Extraia qualquer frase sobre o valor ou preço do produto. Retorne como uma lista Python.

Texto: {review}

Retorne a resposta no formato JSON.
""")

In [17]:
print(review_template.format_messages(review=review_cliente))

[HumanMessage(content='\nPara o texto a seguir, extraia as seguintes informações:\n\npresente: o item foi comprado para alguém? True caso verdadeiro e False caso falso ou não tenha informação.\n\ndias_entrega: quantos dias para a entrega chegar? se a resposta não for encontrada, retorne -1.\n\npercepcao_de_valor: Extraia qualquer frase sobre o valor ou preço do produto. Retorne como uma lista Python.\n\nTexto: \nEste soprador de folhas é bastante incrível. Ele tem quatro configurações: sopro de vela, brisa suave, cidade ventosa e tornado. Chegou em dois dias, bem a tempo para o presente de aniversário da minha esposa. Acho que minha esposa gostanto tanto que ficou sem palavras. Até agora, fui o único a usá-lo, e tenho usando em todas as manhãs alternadas para limpar as folhas do nosso gramado. É um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado, mas acho que vale a penas pelas características extras.\n\n\nRetorne a resposta no formato JSON.\n', additional_

In [18]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI()

In [19]:
resposta = chat.invoke(
    review_template.format_messages(review=review_cliente)
)

In [23]:
import json
print(resposta.content)

{
    "presente": true,
    "dias_entrega": 2,
    "percepcao_de_valor": ["É um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado, mas acho que vale a penas pelas características extras."]
}


## Aplicação de Output Parsers

In [24]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

In [25]:
schema_presente = ResponseSchema(
    name = 'presente',
    type = 'bool',
    description = 'o item foi comprado para alguém? True caso verdadeiro e False caso falso ou não tenha informação.'
)

schema_entrega = ResponseSchema(
    name = 'dias_entrega',
    type = 'int',
    description = 'quantos dias para a entrega chegar? se a resposta não for encontrada, retorne -1.'
)

schema_valor = ResponseSchema(
    name = 'percepcao_de_valor',
    type = 'list',
    description = 'Extraia qualquer frase sobre o valor ou preço do produto. Retorne como uma lista Python.'
)

response_schema = [
    schema_presente,
    schema_entrega,
    schema_valor
]

output_parser = StructuredOutputParser.from_response_schemas(response_schema)

schema_formatado = output_parser.get_format_instructions()

print(schema_formatado)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"presente": bool  // o item foi comprado para alguém? True caso verdadeiro e False caso falso ou não tenha informação.
	"dias_entrega": int  // quantos dias para a entrega chegar? se a resposta não for encontrada, retorne -1.
	"percepcao_de_valor": list  // Extraia qualquer frase sobre o valor ou preço do produto. Retorne como uma lista Python.
}
```


In [26]:
from langchain.prompts import ChatPromptTemplate

review_template = ChatPromptTemplate.from_template(
    """
    Para o texto a seguir, extraia as seguintes informações:

    presente, dias_entrega e percepcao_de_valor

    Texto: {review}

    {schema}
    """,
    partial_variables={'schema': schema_formatado}
)

print(review_template.format_messages(review=review_cliente))

[HumanMessage(content='\n    Para o texto a seguir, extraia as seguintes informações:\n\n    presente, dias_entrega e percepcao_de_valor\n\n    Texto: \nEste soprador de folhas é bastante incrível. Ele tem quatro configurações: sopro de vela, brisa suave, cidade ventosa e tornado. Chegou em dois dias, bem a tempo para o presente de aniversário da minha esposa. Acho que minha esposa gostanto tanto que ficou sem palavras. Até agora, fui o único a usá-lo, e tenho usando em todas as manhãs alternadas para limpar as folhas do nosso gramado. É um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado, mas acho que vale a penas pelas características extras.\n\n\n    The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"presente": bool  // o item foi comprado para alguém? True caso verdadeiro e False caso falso ou não tenha informação.\n\t"dias_entrega": int  // quantos dia

In [27]:
resposta = chat.invoke(review_template.format_messages(review=review_cliente))

In [29]:
print(resposta.content)

```json
{
	"presente": true,
	"dias_entrega": 2,
	"percepcao_de_valor": ["É um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado, mas acho que vale a pena pelas características extras."]
}
```


In [30]:
resposta_json = output_parser.parse(resposta.content)

In [31]:
resposta_json

{'presente': True,
 'dias_entrega': 2,
 'percepcao_de_valor': ['É um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado, mas acho que vale a pena pelas características extras.']}

In [32]:
resposta_json['presente']

True

In [33]:
resposta_json['dias_entrega']

2

In [34]:
resposta_json['percepcao_de_valor']

['É um pouco mais caro do que os outros sopradores de folhas disponíveis no mercado, mas acho que vale a pena pelas características extras.']